In [1]:
import pandas as pd
import numpy as np

In [4]:
# Parametreler
n = 1000  # satır sayısı 
np.random.seed(42)

In [5]:
# Kategorik değişkenler
sectors = ["Enerji", "Gıda", "Teknoloji", "İnşaat", "Sağlık"]
risk_levels = ["Düşük", "Orta", "Yüksek"]

In [6]:
# Temel kolonlar

project_id = np.arange(1, n+1)
sector = np.random.choice(sectors, n)
risk_level = np.random.choice(risk_levels, n, p=[0.3, 0.5, 0.2])
duration_months = np.random.randint(6, 36, n)  # 6-36 ay arası projeler


In [7]:
# Finansal kolonlar

# Yatırım tutarı (lognormal dağılım, TL)
invest_amount = np.random.lognormal(mean=12, sigma=0.5, size=n)

# Beklenen getiri oranı, risk seviyesine göre
expected_return_rate = []
for r in risk_level:
    if r == "Düşük":
        expected_return_rate.append(np.random.normal(0.06, 0.01))
    elif r == "Orta":
        expected_return_rate.append(np.random.normal(0.10, 0.02))
    else:
        expected_return_rate.append(np.random.normal(0.15, 0.03))
expected_return_rate = np.clip(expected_return_rate, 0, 0.30)

# Başarı olasılığı, risk seviyesine göre
success_prob = []
for r in risk_level:
    if r == "Düşük":
        success_prob.append(np.random.uniform(0.85, 0.95))
    elif r == "Orta":
        success_prob.append(np.random.uniform(0.65, 0.85))
    else:
        success_prob.append(np.random.uniform(0.40, 0.70))
success_prob = np.array(success_prob)

# --------------------------
# Pay oranları (banka / yatırımcı)
# --------------------------
bank_share_ratio = np.round(np.random.uniform(0.3, 0.7, n), 2)
investor_share_ratio = 1 - bank_share_ratio

In [8]:
# Gerçekleşme simülasyonu
# --------------------------
success_flag = (np.random.rand(n) < success_prob).astype(int)

actual_return_rate = []
for i in range(n):
    if success_flag[i] == 1:
        # Başarılı projelerde beklenenin yakınında
        actual_return_rate.append(expected_return_rate[i] + np.random.normal(0, 0.02))
    else:
        # Başarısız projelerde negatif getiri
        actual_return_rate.append(-np.random.uniform(0.05, 0.20))
actual_return_rate = np.round(actual_return_rate, 3)

# --------------------------
# Kâr/Zarar hesaplama
# --------------------------
bank_profit_loss = invest_amount * actual_return_rate * bank_share_ratio
investor_profit_loss = invest_amount * actual_return_rate * investor_share_ratio

In [9]:
# DataFrame oluştur
# --------------------------
df = pd.DataFrame({
    "project_id": project_id,
    "sector": sector,
    "invest_amount": np.round(invest_amount, 2),
    "duration_months": duration_months,
    "risk_level": risk_level,
    "expected_return_rate": np.round(expected_return_rate, 3),
    "success_prob": np.round(success_prob, 3),
    "bank_share_ratio": bank_share_ratio,
    "investor_share_ratio": investor_share_ratio,
    "success_flag": success_flag,
    "actual_return_rate": actual_return_rate,
    "bank_profit_loss": np.round(bank_profit_loss, 2),
    "investor_profit_loss": np.round(investor_profit_loss, 2)
})

In [11]:
# Kontrol & kayıt
# --------------------------
print(df.head())
df.to_csv("profit_sharing_data.csv", index=False)
print("✅ Sentetik veri başarıyla oluşturuldu: profit_sharing_data.csv")

   project_id     sector  invest_amount  duration_months risk_level  \
0           1     İnşaat      244104.53               16     Yüksek   
1           2     Sağlık      212689.10                6       Orta   
2           3  Teknoloji      245073.96               18       Orta   
3           4     Sağlık       89098.54               24     Yüksek   
4           5     Sağlık      218374.09               29       Orta   

   expected_return_rate  success_prob  bank_share_ratio  investor_share_ratio  \
0                 0.130         0.566              0.37                  0.63   
1                 0.092         0.698              0.44                  0.56   
2                 0.096         0.735              0.48                  0.52   
3                 0.157         0.690              0.33                  0.67   
4                 0.099         0.782              0.53                  0.47   

   success_flag  actual_return_rate  bank_profit_loss  investor_profit_loss  
0       